In [1]:
import pandas as pd, json, requests
from time import sleep

In [2]:
# Definindo os parâmetros de consulta do CNPJ, esses dados você consegue encontrar no próprio site da API
def consulta_cnpj(cnpj):
    n_cnpj = str(cnpj)
    url = (f'https://receitaws.com.br/v1/cnpj/{cnpj}')
    querystring = {'token':"XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX", 'cnpj':'06990590000123', 'plugin':'RF'}
    response = requests.request('GET', url, params=querystring, headers={'Cache-Control': 'no-cache'})
    resp = json.loads(response.text)
    
    
    # Uma verificação simples com IF caso CNPJ não seja válido; se for válido, vai retornar apenas os dados selecionados abaixo
    if resp.get('status') == 'ERROR':
        return {'cnpj':n_cnpj, 'situação CNPJ':'CNPJ Inválido'}
    else:
        return  {'cnpj':resp['cnpj'], 'situação CNPJ':resp['situacao'], 'nome empresarial':resp['nome'], 'pais':'Brasil',
                 'tipo logradouro':'', 'logradouro':resp['logradouro'], 'numero':resp['numero'], 
                 'complemento':resp['complemento'], 'bairro':resp['bairro'], 'municipio':resp['municipio'],
                 'cep':resp['cep'], 'uf':resp['uf']}
    

In [3]:
# Formatando o CNPJ e excluindo qualquer linha que não esteja com CNPJ
base = pd.read_excel('cnpjs.xlsx', usecols='A:B')
base = base[base.CNPJ != '#N/D']
base = base[base.CNPJ != '0']
base = base[base.CNPJ.notnull()]

base['CNPJ'] = base['CNPJ'].apply(str)
base['CNPJ'] = base['CNPJ'].str.replace(r'[^0-9]+', '')

base

C:\Users\gporto\AppData\Local\Temp\ipykernel_20628\2957604221.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  base['CNPJ'] = base['CNPJ'].str.replace(r'[^0-9]+', '')


,nome da empresa,CNPJ
0,Assembleia Legislativa do Estado de Minas Gerais,17516113000147
1,Tribunal de Contas do Estado de Minas Gerais,21154877000107
2,Fundo de Apoio Habitacional da Assembleia Legi...,17516113000147
3,Defensoria Pública do Estado de Minas Gerais,05599094000180
4,Polícia Civil do Estado de Minas Gerais,18715532000170


In [4]:
# Clever solution para guardar dict com mais de um key-value em data frame, só colocar o dict dentro de colchetes. Assim o 
# python entende como uma lista de keys e values a serem armazenadas como colunas e linhas
# https://stackoverflow.com/questions/18837262/convert-python-dict-into-a-dataframe

df = pd.DataFrame()
n = 0

for i in base['CNPJ']:
    
    retorno = pd.DataFrame.from_dict([consulta_cnpj(i)])
    n += 1
    
    df = pd.concat([df, retorno]) 
    df.to_excel('cnpjs consultados.xlsx', index=False)

    
    # O sleep aqui tem uma função muito simples. A versão grátis da API da Receita Federal só permite 3 consultas por minuto.
    # Ao invés de trabalhar com máscara de IP ou gambiarras mais complicadas, o sleep já resolve fazendo com que o script
    # consulta apenas 3 CNPJs a cada minuto.
    if n % 3 == 0:
        sleep(60)  
              
df

,cnpj,situação CNPJ,nome empresarial,pais,tipo logradouro,logradouro,numero,complemento,bairro,municipio,cep,uf
0,17.516.113/0001-47,ATIVA,ASSEMBLEIA LEGISLATIVA DO ESTADO DE MINAS GERAIS,Brasil,,R RODRIGUES CALDAS,30,,SANTO AGOSTINHO,BELO HORIZONTE,30.190-921,MG
0,21.154.877/0001-07,ATIVA,TRIBUNAL DE CONTAS DO ESTADO DE MINAS GERAIS,Brasil,,AV RAJA GABAGLIA,1315,,LUXEMBURGO,BELO HORIZONTE,30.380-103,MG
0,17.516.113/0001-47,ATIVA,ASSEMBLEIA LEGISLATIVA DO ESTADO DE MINAS GERAIS,Brasil,,R RODRIGUES CALDAS,30,,SANTO AGOSTINHO,BELO HORIZONTE,30.190-921,MG
0,05.599.094/0001-80,ATIVA,DEFENSORIA PUBLICA DO ESTADO DE MINAS GERAIS,Brasil,,R DOS GUAJAJARAS,1707,,BARRO PRETO,BELO HORIZONTE,30.180-101,MG
0,18.715.532/0001-70,ATIVA,POLICIA CIVIL DO ESTADO DE MINAS GERAIS,Brasil,,ROD PAPA JOAO PAULO II,4143,ANDAR 4 EDIFICIO MINAS,SERRA VERDE,BELO HORIZONTE,31.630-900,MG


In [5]:
# Manipulação simples da formatação do CNPJ para fazer um map com dict do CNPJ e nome da empresa pra gerar o .xlsx retorno
df.insert(0, column='cnpj_dig', value=df['cnpj'].str.replace(r'[^0-9]+', ''))

mapping = dict(base[['CNPJ', 'nome da empresa']].values)
df.insert(0, column='nome da empresa', value=df.cnpj_dig.map(mapping))

df.drop('cnpj_dig', axis=1, inplace=True)
df.to_excel('cnpjs consultados.xlsx', index=False)

C:\Users\gporto\AppData\Local\Temp\ipykernel_20628\195680308.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.insert(0, column='cnpj_dig', value=df['cnpj'].str.replace(r'[^0-9]+', ''))
